## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp (F),% Humidity,% Cloudiness,Wind Speed (mph),Current Description
0,0,Norilsk,RU,69.3535,88.2027,-32.37,79,37,7.05,scattered clouds
1,1,Camacupa,AO,-12.0167,17.4833,60.84,84,100,4.25,overcast clouds
2,2,Hithadhoo,MV,-0.6000,73.0833,81.97,74,66,21.23,broken clouds
3,3,Pevek,RU,69.7008,170.3133,-18.40,88,99,0.94,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,51.80,76,40,4.61,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximun temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp (F)"] >= min_temp) & (city_data_df["Max Temp (F)"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp (F),% Humidity,% Cloudiness,Wind Speed (mph),Current Description
2,2,Hithadhoo,MV,-0.6000,73.0833,81.97,74,66,21.23,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.76,76,92,12.82,overcast clouds
9,9,Mataura,NZ,-46.1927,168.8643,78.93,42,10,3.51,clear sky
13,13,Mudgee,AU,-32.5943,149.5871,90.00,32,0,11.50,clear sky
16,16,Aracataca,CO,10.5918,-74.1898,75.99,82,52,0.72,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

136


City_ID                136
City                   136
Country                136
Lat                    136
Lng                    136
Max Temp (F)           136
% Humidity             136
% Cloudiness           136
Wind Speed (mph)       136
Current Description    136
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

136


,City_ID,City,Country,Lat,Lng,Max Temp (F),% Humidity,% Cloudiness,Wind Speed (mph),Current Description
2,2,Hithadhoo,MV,-0.6000,73.0833,81.97,74,66,21.23,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.76,76,92,12.82,overcast clouds
9,9,Mataura,NZ,-46.1927,168.8643,78.93,42,10,3.51,clear sky
13,13,Mudgee,AU,-32.5943,149.5871,90.00,32,0,11.50,clear sky
16,16,Aracataca,CO,10.5918,-74.1898,75.99,82,52,0.72,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp (F)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,81.97,broken clouds,-0.6000,73.0833,
8,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,
9,Mataura,NZ,78.93,clear sky,-46.1927,168.8643,
13,Mudgee,AU,90.00,clear sky,-32.5943,149.5871,
16,Aracataca,CO,75.99,broken clouds,10.5918,-74.1898,
22,Bundaberg,AU,86.00,scattered clouds,-24.8500,152.3500,
24,Kahului,US,82.40,scattered clouds,20.8947,-156.4700,
35,Kapaa,US,77.00,light rain,22.0752,-159.3190,
40,Butaritari,KI,81.52,scattered clouds,3.0707,172.7902,
53,Lorengau,PG,85.96,overcast clouds,-2.0226,147.2712,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,81.97,broken clouds,-0.6000,73.0833,Scoop Guest House
8,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,Pension Maro'i
9,Mataura,NZ,78.93,clear sky,-46.1927,168.8643,Ellie's Villa
13,Mudgee,AU,90.00,clear sky,-32.5943,149.5871,Parklands Resort & Conference Centre
16,Aracataca,CO,75.99,broken clouds,10.5918,-74.1898,Hostal de los buendia
22,Bundaberg,AU,86.00,scattered clouds,-24.8500,152.3500,Takalvan Motel
24,Kahului,US,82.40,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
35,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
40,Butaritari,KI,81.52,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
53,Lorengau,PG,85.96,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp (F)</dt><dd>{Max Temp (F)} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))